In [68]:
# Libraries import
import math as m
import pygame
from pygame.draw import *
from random import *
import time
pygame.init()

# Params

# Auxilary
ball_list = []
rocket_list = []

# Screen
FPS = 24
screen_width = 1500
screen_hight = 800
screen = pygame.display.set_mode((screen_width, screen_hight))

# Graphiс data
general_color_list =  [(0, 0, 0), # Black
                        (127, 255, 212), # Aquamarine
                        (192, 192, 192), # Silver
                        (255, 0, 0), # Red
                        (0, 0, 255), # Blue
                        (0, 255, 255), # Cyan
                        (255, 140, 0), # Dark orange
                        (30, 144, 255), # Fake blue
                        (0, 128, 0), # Green
                        (124, 252, 0), # Green grass
                        (40, 200, 170), # Green spring
                        (240, 230, 140), # Haki
                        (219, 112, 147), # Heavy pink
                        (75, 0, 130), # Indigo
                        (244, 255, 255), # Light blue
                        (245, 245, 220), # Light white
                        (255, 0, 255), # Magenta
                        (25, 25, 112), # Midnight blue
                        (128, 0, 128), # Purple
                        (255, 20, 100), # Pink
                        (255, 255, 0), # Yellow
                       ]

# General objects
objects_amount = 5

# Balls
max_ball_radius = 50
max_ball_speed = 10
min_ball_radius = 25
min_ball_speed = 5

# Rockets
body_x_ratio = 0.5
body_y_ratio = 0.5
max_rocket_hight = 100
max_rocket_speed = 34
min_rocket_hight = 50
min_rocket_speed = 17
rocket_generation_chance = 20
rocket_hight_width_ratio = 2
score_per_rocket = 10
rocket_time_hold = 1 # Time without rocket's generation

# Flow control
clock = pygame.time.Clock()
finished = False
start_time = time.time()

# User interface
game_time = 20
leader_amount = 5
score = 0

def count_direction(latest_direction: int, radius: int, x:int, y: int):
    '''
    Counts speed direction after reflection
    latest_direction is direction before reflection
    radius is radius of ball in pixels
    x is x coordinate of ball center
    y is y coordinate of ball center
    '''
    
    direction = latest_direction
    if x < radius:
        direction = random() * m.pi - m.pi / 2
    if x > screen_width - radius:
        direction = random() * m.pi + m.pi / 2
    if y < radius:
        direction = random() * m.pi - m.pi
    if y > screen_hight - radius:
        direction = random() * m.pi
    return direction
    
    
def count_rocket_score(rocket: list, rocket_list: list, click_x: int, click_y: int):
    '''
    Checks if click has occured inside the rocket
    rocket is list with rocket params
    rocket_list is list with all rockets in the game
    click_x is x coordinate of mouse click
    click_y is y coordinate of mouse click
    '''
    
    hight = rocket[1]
    width = rocket[3]
    x = rocket[4]
    y = rocket[5]
    if click_x >= x and click_x <= x + width and click_y >= y and click_y <= y + hight:
        delta_score = score_per_rocket
        rocket_list.remove(rocket)
    else:
        delta_score = 0
    new_score = score + delta_score
    return new_score
    
    
def count_score(ball: list, ball_list: list, click_x: int, click_y: int, rocket_list: list):
    '''
    Checks if click has occured inside the circle
    ball is list with ball params
    ball_list is list of all balls in game
    click_x is x coordinate of mouse click
    click_y is y coordinate of mouse click
    rocket_list is list of all rockets in game
    '''
    
    radius = ball[1]
    x = ball[4]
    y = ball[5]
    distance = get_distance(x, y, click_x, click_y)
    if distance <= radius:
        delta_score = 1 # Increace score by one
        ball_list.remove(ball)
        generate_object(1, ball_list, rocket_list) # Generate 1 ball
    else:
        delta_score = 0 # Don't increace score
    new_score = score + delta_score
    return new_score


def check_time():
    '''
    Checks if the game time is over
    '''
    
    
    if time.time() - start_time > game_time:
        return True
    else:
        return False
    
    
def get_distance(x_1: int, y_1: int, x_2: int, y_2: int):
    '''
    Gets distance between 2 points
    x_1 is x coordinate of 1 point
    y_1 is y coordinate of 1 point
    x_2 is x coordinate of 2 point
    y_2 is y coordinate of 2 point
    '''
    
    distance = m.sqrt((x_1-x_2) ** 2 + (y_1-y_2) ** 2)
    return distance
    
def generate_rocket(rocket_list: list):
    '''
    Generates a rocket on screen
    rocket_list is list of all rockets
    '''
    
    color = general_color_list[randint(1, len(general_color_list) - 1)] # Rocket color from list but not black
    hight = randint(min_rocket_hight, max_rocket_hight)
    speed = randint(min_rocket_speed, max_rocket_speed)
    width = int(hight / rocket_hight_width_ratio)
    x = randint(0, screen_width - width) # Random x coordinate to draw all image on screen
    y = screen_hight
    rocket_list.append([color, hight, speed, width, x, y])
    
    
def generate_object(objects_amount: int, ball_list: list, rocket_list: list):
    '''
    Generates objects on the screen
    objects_amount is amount of objects to generate
    ball_list is list of all balls in game
    rocket_list is list of all rockets in game
    '''
    
    if rocket_needed():
        generate_rocket(rocket_list)
    for ball in range(objects_amount):
        color = general_color_list[randint(1, len(general_color_list) - 1)] # Ball color from list but not black
        radius = randint(min_ball_radius, max_ball_radius)
        speed_direction = 2 * m.pi * random() # Speed direction from 0 to 360 degrees
        speed = randint(min_ball_speed, max_ball_speed)
        x = randint(radius, screen_width - radius)
        y = randint(radius, screen_hight - radius)
        ball_list.append([color, radius, speed_direction, speed, x, y])
    
    
def new_ball(color: list, radius: int, x:int, y: int):
    '''
    Draws new ball
    color is list of 3 RGB values
    radius is radius of ball
    x is x coordinate of ball center
    y is y coordinate of ball center
    '''

    circle(screen, color, (x, y), radius)
    
    
def new_rocket(color: list, hight: int, width: int, x: int, y: int):
    '''
    Daws new rocket
    color is list of 3 RGB values
    hight is hight of rocket in pixels
    width is width of rocket in pixels
    zero-point is upper-left point of rect inside witch rocket is drawn
    x is x coordinate of zero-point
    y is y coordinate of zero-point
    '''
    
    # Draw body
    rect(screen, color, (x + int(width * (1 - body_x_ratio) / 2), y + int(hight * (1 - body_y_ratio) / 2),
                         int(width * body_x_ratio), int(hight * body_y_ratio)))
    
    # Draw nose
    polygon(screen, color, ((x + width // 2, y),
                            (x + int(width * (1 + body_x_ratio) / 2), y + int(hight * (1 - body_y_ratio) / 2)),
                            (x + int(width * (1 - body_x_ratio) / 2), y + int(hight * (1 - body_y_ratio) / 2))))
    
    # Draw window
    window_color = general_color_list[1] # Aquamarine
    x_radius = width * body_x_ratio // 4
    y_radius = hight * body_y_ratio // 8
    radius = int(min(x_radius, y_radius))
    circle(screen, window_color, (x + width // 2, y + int(hight * (2 - body_y_ratio) / 4)), radius)
    
    # Draw stabilizers
    polygon(screen, color, ((x + int(width * (1 - body_x_ratio) / 2), y + hight // 2),
                            (x + int(width * (1 + body_x_ratio) / 2), y + hight // 2),
                            (x + width, y + int(hight * (1 + body_y_ratio) / 2)),
                            (x, y + int(hight * (1 + body_y_ratio) / 2))))
    
    # Draw engine
    engine_color = general_color_list[2] # Silver
    polygon(screen, engine_color, ((x + int(width * (2 - body_x_ratio) / 4), y + int(hight * (1 + body_y_ratio) / 2)),
                                   (x + int(width * (2 + body_x_ratio) / 4), y + int(hight * (1 + body_y_ratio) / 2)),
                                   (x + int(width * (1 + body_x_ratio) / 2), y + int(hight * (3 + body_y_ratio) / 4)),
                                   (x + int(width * (1 - body_x_ratio) / 2), y + int(hight * (3 + body_y_ratio) / 4))))
    
    # Draw fire
    fire_color = general_color_list[3] # Red
    polygon(screen, fire_color, ((x + int(width * (1 - body_x_ratio) / 2), y + int(hight * (3 + body_y_ratio) / 4)),
                                   (x + int(width * (1 + body_x_ratio) / 2), y + int(hight * (3 + body_y_ratio) / 4)),
                                   (x + width // 2, y + hight)))


def move_ball(ball: list):
    '''
    Moves ball with wall reflections
    ball is list with ball params
    '''
    
    color = ball[0]
    direction = ball[2]
    radius = ball[1]
    speed = ball[3]
    x = ball[4]
    y = ball[5]
    x = int(x + speed * m.cos(direction))
    y = int(y - speed * m.sin(direction))
    direction = count_direction(direction, radius, x, y)
    new_ball(color, radius, x, y)
    ball[2] = direction
    ball[4] = x
    ball[5] = y
    
    
def move_rocket(rocket: list, rocket_list: list):
    '''
    Flies rocket over the screen
    rocket is list with rocket params
    rocket_list is list of all rockets in game
    '''
    
    color = rocket[0]
    hight = rocket[1]
    speed = rocket[2]
    width = rocket[3]
    x = rocket[4]
    y = rocket[5]
    y = int(y - speed)
    if y < - hight:
        rocket_list.remove(rocket)
    new_rocket(color, hight, width, x, y)
    rocket[5] = y
    
def rocket_needed():
    '''
    Returns 'True' if rocket is needed to generate
    '''
    
    rocket_generation = randint(1, 100) # Random chance generation
    if rocket_generation <= rocket_generation_chance and time.time() - start_time > rocket_time_hold:
        return True
    else:
        return False

def set_leaderboard(score: int):
    '''
    Sves player result to the leaderboard
    score is score amount of player
    '''
    
    leader_list = []
    output = open('leaderboard.txt', 'r')
    for line in output:
        line = line.strip()
        leader_list.append(line)
    output.close()
    leader_list = sort_leaderboard(leader_list)
    output = open('leaderboard.txt', 'w')
    for leader in leader_list:
        print(leader, file = output)
    output.close()
    

def sort_leaderboard(leader_list: list):
    '''
    Sorts leaderboard
    leader_list is list to sort
    '''
    leader_data = []
    leader_score = []
    current_time = time.asctime()
    leader_list.append('Time -> '+str(current_time)+'; score -> '+str(score))
    for leader in leader_list:
        leader_score.append(int(leader[43:])) # Part of string with score as int number
        leader_data.append(leader[:43]) # Another part of string
    for i in range(len(leader_score) - 1): # Sort
        for j in range(i+1, len(leader_score)):
            if leader_score[j] > leader_score[i]:
                swap = leader_score[i]
                leader_score[i] = leader_score[j]
                leader_score[j] = swap
                swap = leader_data[i]
                leader_data[i] = leader_data[j]
                leader_data[j] = swap
    leader_list = []
    for i in range(len(leader_data)):
        leader_list.append(leader_data[i]+str(leader_score[i]))
    leader_list = leader_list[0:leader_amount]
    return leader_list
    

generate_object(objects_amount, ball_list, rocket_list)
while not finished:
    clock.tick(FPS)
    for ball in ball_list:
        move_ball(ball)
    for rocket in rocket_list:
        move_rocket(rocket, rocket_list)
    pygame.display.update()
    screen.fill(general_color_list[0])
    finished = check_time()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            finished = True
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for ball in ball_list:
                score = count_score(ball, ball_list, event.pos[0], event.pos[1], rocket_list)
            for rocket in rocket_list:
                score = count_rocket_score(rocket, rocket_list, event.pos[0], event.pos[1])
pygame.quit()
set_leaderboard(score)